In [ ]:
file = "/FileStore/tables/travel_insurance.csv"

In [ ]:
from pyspark import SparkContext

In [ ]:
sc = SparkContext.getOrCreate()

In [ ]:
df = spark.read.format('csv').option('header', 'true').option('inferschema', 'true').load(file)

In [ ]:
df.show(truncate = 27)

+------+-------------+--------------------+---------------------------+-----+--------+---------------------------+---------+--------------------+------+---+
|Agency|  Agency Type|Distribution Channel|               Product Name|Claim|Duration|                Destination|Net Sales|Commision (in value)|Gender|Age|
+------+-------------+--------------------+---------------------------+-----+--------+---------------------------+---------+--------------------+------+---+
|   CBH|Travel Agency|             Offline|         Comprehensive Plan|   No|     186|                   MALAYSIA|    -29.0|                9.57|     F| 81|
|   CBH|Travel Agency|             Offline|         Comprehensive Plan|   No|     186|                   MALAYSIA|    -29.0|                9.57|     F| 71|
|   CWT|Travel Agency|              Online|Rental Vehicle Excess In...|   No|      65|                  AUSTRALIA|    -49.5|                29.7|  null| 32|
|   CWT|Travel Agency|              Online|Rental Vehicle 

In [ ]:
df = df.withColumnRenamed('Commision (in value)', 'Commission')

In [ ]:
df.show(truncate = 27)

+------+-------------+--------------------+---------------------------+-----+--------+---------------------------+---------+----------+------+---+
|Agency|  Agency Type|Distribution Channel|               Product Name|Claim|Duration|                Destination|Net Sales|Commission|Gender|Age|
+------+-------------+--------------------+---------------------------+-----+--------+---------------------------+---------+----------+------+---+
|   CBH|Travel Agency|             Offline|         Comprehensive Plan|   No|     186|                   MALAYSIA|    -29.0|      9.57|     F| 81|
|   CBH|Travel Agency|             Offline|         Comprehensive Plan|   No|     186|                   MALAYSIA|    -29.0|      9.57|     F| 71|
|   CWT|Travel Agency|              Online|Rental Vehicle Excess In...|   No|      65|                  AUSTRALIA|    -49.5|      29.7|  null| 32|
|   CWT|Travel Agency|              Online|Rental Vehicle Excess In...|   No|      60|                  AUSTRALIA|    

In [ ]:
from pyspark.sql.functions import desc
counts = df.groupBy('Destination').count()

counts.orderBy(desc('count')).show(10)

+-------------+-----+
|  Destination|count|
+-------------+-----+
|    SINGAPORE|13255|
|     MALAYSIA| 5930|
|     THAILAND| 5894|
|        CHINA| 4796|
|    AUSTRALIA| 3694|
|    INDONESIA| 3452|
|UNITED STATES| 2530|
|  PHILIPPINES| 2490|
|    HONG KONG| 2411|
|        INDIA| 2251|
+-------------+-----+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import mean
mean_age = df.groupBy('Claim').agg(mean('Age'))

In [ ]:
mean_age.show()

+-----+------------------+
|Claim|          avg(Age)|
+-----+------------------+
|   No|39.989823554864664|
|  Yes| 38.63430420711974|
+-----+------------------+



In [ ]:
df.groupBy('Agency Type').sum('Commission').show()

+-------------+------------------+
|  Agency Type|   sum(Commission)|
+-------------+------------------+
|Travel Agency|350183.16000002477|
|     Airlines|271044.38000001264|
+-------------+------------------+



In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

## predictors: Agency Type, Distribution Channel, Duration, Net Sales, Commission, Gender, Age
predictors = ['Agency Type', 'Distribution Channel', 'Duration', 'Net Sales', 'Commission', 'Gender', 'Age']
##  Use Claim as the response variable
response = ['Claim']

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<command-619884183136599> in <cell line: 9>()
      7 response = ['Claim']
      8 
----> 9 filtered_df = df.na.drop()

AttributeError: 'StringIndexerModel' object has no attribute 'na'

In [ ]:
df.show(5)

+------+-------------+--------------------+--------------------+-----+--------+-----------+---------+----------+------+---+
|Agency|  Agency Type|Distribution Channel|        Product Name|Claim|Duration|Destination|Net Sales|Commission|Gender|Age|
+------+-------------+--------------------+--------------------+-----+--------+-----------+---------+----------+------+---+
|   CBH|Travel Agency|             Offline|  Comprehensive Plan|   No|     186|   MALAYSIA|    -29.0|      9.57|     F| 81|
|   CBH|Travel Agency|             Offline|  Comprehensive Plan|   No|     186|   MALAYSIA|    -29.0|      9.57|     F| 71|
|   CWT|Travel Agency|              Online|Rental Vehicle Ex...|   No|      65|  AUSTRALIA|    -49.5|      29.7|  null| 32|
|   CWT|Travel Agency|              Online|Rental Vehicle Ex...|   No|      60|  AUSTRALIA|    -39.6|     23.76|  null| 32|
|   CWT|Travel Agency|              Online|Rental Vehicle Ex...|   No|      79|      ITALY|    -19.8|     11.88|  null| 41|
+------+

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

# create a list of predictor variables and response variable
predictors = ['Agency Type', 'Distribution Channel', 'Duration', 'Net Sales', 'Commission', 'Gender', 'Age']
response = ['Claim']

# filter out rows with missing data
filtered_df = df.na.drop()

# convert string columns to integer using StringIndexer
string_indexers = [StringIndexer(inputCol=column, outputCol=column+"_indexed") for column in ['Agency Type', 'Distribution Channel', 'Gender']]
string_indexer_pipeline = Pipeline(stages=string_indexers)
indexed_df = string_indexer_pipeline.fit(filtered_df).transform(filtered_df)

# create dummy variables using OneHotEncoder
onehot_encoders = [OneHotEncoder(inputCol=column+"_indexed", outputCol=column+"_encoded") for column in ['Agency Type', 'Distribution Channel', 'Gender']]
onehot_encoder_pipeline = Pipeline(stages=onehot_encoders)
encoded_df = onehot_encoder_pipeline.fit(indexed_df).transform(indexed_df)

# assemble feature vector with predictor variables
assembler = VectorAssembler(inputCols=[column+"_encoded" if column in ['Agency Type', 'Distribution Channel', 'Gender'] else column for column in predictors], outputCol="features")
transformed_df = assembler.transform(encoded_df).select('features', *response)

indexer = StringIndexer(inputCol='Claim', outputCol='label')
index_model = indexer.fit(transformed_df)
input = index_model.transform(transformed_df).select("features", "label")
input.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.0,0.0,186.0,-2...|  0.0|
|[0.0,0.0,186.0,-2...|  0.0|
|[1.0,1.0,66.0,-12...|  0.0|
|[1.0,1.0,1.0,-18....|  0.0|
|[0.0,1.0,53.0,-13...|  0.0|
+--------------------+-----+
only showing top 5 rows



In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# split data into training and test sets
(training_data, test_data) = input.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# create a logistic regression object
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

lr = lr.fit(training_data)

In [ ]:
pred = lr.transform(test_data)

In [ ]:
pred.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(7,[2,3,6],[364.0...|  0.0|[3.04358499837398...|[0.95450476318812...|       0.0|
|(7,[2,3,6],[364.0...|  0.0|[3.60739507725494...|[0.97359379318160...|       0.0|
|(7,[2,3,6],[364.0...|  0.0|[3.02009718061343...|[0.95347383675103...|       0.0|
|(7,[2,3,6],[364.0...|  0.0|[3.30290370414360...|[0.96452829062769...|       0.0|
|(7,[2,3,6],[365.0...|  0.0|[3.13946546460185...|[0.95849161918617...|       0.0|
|(7,[2,3,6],[365.0...|  0.0|[3.30160485531530...|[0.96448382568413...|       0.0|
|(7,[2,3,6],[366.0...|  0.0|[3.57651672724531...|[0.97278822776306...|       0.0|
|(7,[2,3,6],[368.0...|  0.0|[3.16384957046995...|[0.95945097836644...|       0.0|
|(7,[2,3,6],[375.0...|  0.0|[3.33103734556177...|[0.96547836103253...|       0.0|
|(7,[2,3,6],[384

In [ ]:
prediction_rdd = pred.select("prediction", "label").rdd.map(tuple)
prediction_rdd.take(5)

Out[102]: [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]

In [ ]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

metrics = BinaryClassificationMetrics(prediction_rdd)
# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

Area under PR = 0.03808457014841781
Area under ROC = 0.5


In [ ]:
pred_train = lr.transform(training_data)

prediction_train_rdd = pred_train.select("prediction", "label").rdd.map(tuple)
prediction_train_rdd.take(5)

Out[105]: [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics

metrices_train = MulticlassMetrics(prediction_train_rdd)
metrices_train.accuracy

/databricks/spark/python/pyspark/sql/context.py:165: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
Out[106]: 0.9654560349535772